# Трендовые признаки (нормализованные для Global ML Model)

**Использует модуль** - все функции импортируются из `features.trend_features`

## Вычисляемые признаки:
- **Относительные** расстояния до MA (dist_to_sma_*, dist_to_ema_*)
- Нормализованные наклоны MA
- Momentum (log return за период)
- RSI (уже нормализован 0-100)
- Сигнал и сила тренда

**Полный pipeline**: см. `06_feature_aggregator.ipynb`


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import warnings
warnings.filterwarnings("ignore")

# Добавляем путь к корневой директории ML для импорта модулей
sys.path.insert(0, str(Path.cwd().parent))

# Импорт модуля трендовых признаков
from features.trend_features import (
    build_trend_features,
    TREND_FEATURE_COLUMNS
)

print("✅ Библиотеки и модули загружены")


## Загрузка данных


In [ ]:
# Получаем текущую рабочую директорию
current_dir = Path.cwd()
print(f"Текущая директория: {current_dir}")

# Строим путь к данным относительно текущей директории
DATA_DIR = current_dir.parent.parent / "ML" / "data" / "processed"
OUTPUT_DIR = Path("data") / "features"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"Путь к данным: {DATA_DIR}")

# Проверяем существование пути
if DATA_DIR.exists():
    print("Директория найдена!")
else:
    print("Директория не найдена, проверьте путь")

# Список всех доступных тикеров включая индекс MOEX
TICKERS = [
    "AFKS", "AFLT", "ALRS", "BELU", "BSPB", "CHMF", "FIVE", "GAZP", "GMKN", "HYDR",
    "IMOEX",  # Индекс Мосбиржи
    "IRAO", "LENT", "LKOH", "MAGN", "MGNT", "MTSS", "NLMK", "NVTK", "OZON",
    "PIKK", "PLZL", "ROSN", "RTKM", "SBER", "SNGS", "TATN", "TCSG", "VKCO", "VTBR", "YNDX"
]

print(f"✅ Загрузим {len(TICKERS)} тикеров")
print(f"Тикеры: {TICKERS}")
print(f"\nВычисляемые признаки: {TREND_FEATURE_COLUMNS}")


## Расчет для всех тикеров


In [ ]:
# Загружаем и обрабатываем все тикеры
results = {}

for ticker in TICKERS:
    try:
        print(f"🔄 Обработка {ticker}...", end=" ")
        
        # 1. Загрузка данных
        input_path = DATA_DIR / f"{ticker}_ohlcv_returns.parquet"
        if not input_path.exists():
            print(f"⚠️ Файл не найден: {input_path}")
            continue
        
        df = pd.read_parquet(input_path)
        
        # 2. Расчет трендовых признаков через модуль
        trend_features = build_trend_features(df)
        
        # 3. Объединяем исходные данные с признаками
        df_result = pd.concat([df, trend_features], axis=1)
        
        # 4. Сохранение в словарь результатов
        results[ticker] = df_result
        print(f"✅ Готово ({len(df_result)} строк, {len(trend_features.columns)} признаков)")
        
    except Exception as e:
        print(f"\n❌ Ошибка при обработке {ticker}: {e}")

print("-" * 50)
print(f"🎉 Обработка завершена. Успешно обработано: {len(results)} из {len(TICKERS)}")


## Пример и диагностика


In [ ]:
# Демонстрация на примере SBER
sample_ticker = "SBER"
if sample_ticker in results:
    df_sample = results[sample_ticker]
    
    print(f"Столбцы: {df_sample.columns.tolist()}")
    
    # Показать данные из середины (где должны быть полные данные)
    print(f"\nПример данных из СЕРЕДИНЫ датасета:")
    mid_idx = len(df_sample) // 2
    display_cols = ["date", "close", "dist_to_sma_20", "dist_to_sma_50", "momentum_10", "rsi_14", "trend_signal", "trend_strength"]
    print(df_sample[display_cols].iloc[mid_idx:mid_idx+10])
    
    # Показать последние данные
    print(f"\nПоследние 10 записей:")
    print(df_sample[display_cols].tail(10))
    
    # Статистика по трендам
    print(f"\n--- СТАТИСТИКА ТРЕНДОВ ---")
    print(f"Всего записей: {len(df_sample)}")
    trend_counts = df_sample["trend_signal"].value_counts().sort_index()
    print(f"\nРаспределение сигналов тренда:")
    print(f"  Downtrend (-1): {trend_counts.get(-1, 0)} ({trend_counts.get(-1, 0)/len(df_sample)*100:.1f}%)")
    print(f"  Sideways (0):   {trend_counts.get(0, 0)} ({trend_counts.get(0, 0)/len(df_sample)*100:.1f}%)")
    print(f"  Uptrend (1):    {trend_counts.get(1, 0)} ({trend_counts.get(1, 0)/len(df_sample)*100:.1f}%)")
    
    # Статистика по признакам
    print(f"\n--- СТАТИСТИКА ПРИЗНАКОВ ---")
    print(f"Средний RSI: {df_sample['rsi_14'].mean():.2f}")
    print(f"Средний dist_to_sma_50: {df_sample['dist_to_sma_50'].mean():.4f}")
    print(f"Средняя сила тренда: {df_sample['trend_strength'].mean():.4f}")
else:
    print(f"Тикер {sample_ticker} не найден в результатах")


## Сохранение результатов


In [ ]:
# Сохранение всех результатов
for ticker, df in results.items():
    output_path = OUTPUT_DIR / f"{ticker}_trend_features.parquet"
    df.to_parquet(output_path, index=False)
    print(f"✅ Сохранено: {output_path}")

print(f"\n🎉 Все данные сохранены в {OUTPUT_DIR}")

# Проверка: убедимся что нет абсолютных MA
forbidden = ["sma_20", "sma_50", "sma_200", "ema_20", "ema_50"]
sample_df = results.get("SBER")
if sample_df is not None:
    found = [col for col in sample_df.columns if col in forbidden]
    if found:
        print(f"\n⚠️ Внимание: найдены абсолютные столбцы: {found}")
    else:
        print(f"\n✅ Проверка пройдена: нет абсолютных MA колонок")
